<DIV ALIGN=CENTER>

# Introduction to Logistic Regression
## Professor Robert J. Brunner
  
</DIV>  
-----
-----

## Introduction

In earlier lessons, we have seen how to perform a regression on input
data to predict a continuous value. In some cases, however, we wish to
predict a categorical value, such as True/False or Yes/No. Traditional
regression methods are not optimal for these problems, since they
are not continuous values. In this case, we need to employ a special
function, known as a [_logit_ function][wlf] that can separate data into two
classes. This type of regression is known as [logistic regression][wlr] (since
you are using a logit function). 

In this notebook, we introduce logistic regression and demonstrate how
to use logistic regression to predict whether a flight is on time or
not. In this notebook, we will specifically use the [logistic regression
functionality][sllr] in the scikit learn library. First we will set up
this Notebook by importing the airline data that we will use for our
Logistic Regression.

-----
[wlr]: https://en.wikipedia.org/wiki/Logistic_regression
[wlf]: https://en.wikipedia.org/wiki/Logit
[sllr]: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [1]:
# We do this to ignore several specific Pandas warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd 

# Change this to read a different file, for example
# /home/data_scientist/rppdm/data/2001.csv in a local Docker container.
#
# Note that the JupyterHub server has data from other years in the raw
# subdirectory.
#
#filename = '/home/data_scientist/rppdm/data/2001.csv'
filename = '/home/data_scientist/data/2001.csv'

# Read select columns for all rows.

ucs = (1, 2, 4, 14, 15, 16, 17, 18)
cnms = ['Month', 'Day', 'dTime', 'aDelay', 'dDelay', 'Depart', 'Arrive', 'Distance']

od = pd.read_csv(filename, header=0, na_values=['NA'], usecols=ucs, names=cnms)

-----

### Data Transformation

Following the discussion in the [Introduction to Machine Learning
Pre-Processing][imlp] notebook, we first perform some data
pre-processing. To simplify the computations, we will first extract only
those flights that depart from O'Hare international airport in Chicago.
After this, we will drop any row that has no value in the _departure
time_ column, and fill any missing values in the _arrival time_ column
with zero.

-----

[imlp]: ../../Week1/notebooks/intro2data.ipynb

In [3]:
alldata = od[od['Depart'] == 'ORD']

# Drop any row that is missing a departure time
#
# axis = 0 means drop rows
# subset=['dTime'] means drop if the departure time is missing
# inplace=True means modify the existing DataFrame

alldata.dropna(axis=0, subset=['dTime'], inplace=True)

# Now replace missing values (which are all in Arrival Delay column)
# with 0, note we could use another value, such as the departure delay.

alldata.fillna(value=0, axis=0, inplace=True)

----

We next extract columns from this cleaned DataFrame for our logistic
regression. For the most flexibility, we extract the hour and minute
columns, and convert the arrival airport column to a categorical value.

-----

In [4]:
# First create a new DataFrame
# For now, simply copy over the columns, but we could
# convert them to integers (for number, the number of minutes in the delay)
# and change data type to save memory.

newdata = alldata[['aDelay', 'dDelay', 'Distance']]

# Now copy over the departure and Arrival columns, 
# but change data type to categoricals. 

#newdata['Depart'] = alldata['Depart'].astype('category')
newdata['Arrive'] = alldata['Arrive'].astype('category')

newdata['Month'] = alldata.Month
newdata['Day'] = alldata.Day
newdata['Hour'] = (alldata.dTime/100.).astype(int)
newdata['Min'] = (alldata.dTime - 100*(alldata.dTime/100.).astype(int)).astype(int)

-----

We now create our categorical column, _arrival late_, that is zero if
the flight arrived less than a specific number of minutes after the
scheduled arrival time (encoded in `value`), or one if it arrived more
than this number of minutes after the scheduled time. We will use this
to train our logistic regressor.

-----

In [5]:
# Add new column for Late Arrival. If arrival delay - departure delay is 
# greater than value, we consider it a late arrival.

value = 5. # Five Minutes
newdata['aLate'] = (newdata.aDelay > value).astype(int)

## Logistic Regression

We will use the scikit learn library to perform logistic regression. As
we have discussed previously, `sklearn` operates on `numpy` arrays.
However, we can use the formula interface to construct a `pandas`
DataFrame, and extract the columns for use in `sklearn`. To do this, we
use the `patsy` library, which supports a formula interface (as we used
with the `statsmodel` library. In this simple example, we will compute a
logistic regression on the delay, distance, month, day, and Hours
columns. We also turn the last three columns into category variables (by
wrapping them in the `C()` notation). In this case, you should try using
only one of these three columns to study the effect of _Month_, _Day_,
or _Hours_ on the logistic regression.

After we create the `numpy` matrices, we next create a
`LogisticRegression` object and use the `fit` method to determine the
best possible regression on the entire data. We use the `score` method
to quantify how well we can do with this technique, achieving nearly
`84%` accuracy.

-----

In [6]:
import numpy as np
import patsy as pts 

y, x = pts.dmatrices('aLate ~ dDelay + Distance + C(Month) + C(Day) + C(Hour)', 
                     newdata, return_type='dataframe')

# y needs to be a 1D array for scikit learn
y = np.ravel(y)

In [7]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model = model.fit(x, y)

print('Score = {0:4.3f} %'.format(model.score(x, y)))

Score = 0.836 %


-----

We can easily change the regression to use only the _Hour_ categorial
column, in addition to the departure delay and distance columns. After
computing this regression, we can explore the different dependence of
the regression formula on these columns by comparing the fit
coefficients as shown below. As one might expect, the coefficients are
very small for hours where few, if any, flights depart. On the other
hand, the coefficients are large for flights leaving around morning,
noon, and early evening.

-----

In [8]:
y, x = pts.dmatrices('aLate ~ dDelay + Distance + C(Hour)', 
                     newdata, return_type='dataframe')

# y needs to be a 1D array for scikit learn
y = np.ravel(y)

model = LogisticRegression()
model = model.fit(x, y)

In [9]:
df = pd.DataFrame({'Variable': x.columns})
df['Coeeficient'] = np.transpose(model.coef_)

print(df)

         Variable   Coeeficient
0       Intercept -7.842642e-01
1    C(Hour)[T.1]  2.225825e-07
2    C(Hour)[T.2]  2.271573e-08
3    C(Hour)[T.3]  2.572224e-09
4    C(Hour)[T.5]  1.643302e-01
5    C(Hour)[T.6] -7.826077e-03
6    C(Hour)[T.7]  4.659641e-02
7    C(Hour)[T.8]  1.015632e-01
8    C(Hour)[T.9] -5.560310e-02
9   C(Hour)[T.10] -4.541367e-02
10  C(Hour)[T.11] -2.452811e-01
11  C(Hour)[T.12]  2.867560e-01
12  C(Hour)[T.13]  4.905729e-02
13  C(Hour)[T.14] -2.433214e-01
14  C(Hour)[T.15]  8.056267e-02
15  C(Hour)[T.16] -1.476660e-01
16  C(Hour)[T.17]  3.166419e-01
17  C(Hour)[T.18] -1.144096e-01
18  C(Hour)[T.19] -1.757001e-02
19  C(Hour)[T.20] -3.264758e-01
20  C(Hour)[T.21] -1.196160e-01
21  C(Hour)[T.22] -4.795377e-01
22  C(Hour)[T.23] -2.008258e-02
23  C(Hour)[T.24]  6.868646e-08
24         dDelay  1.440725e-01
25       Distance  1.057563e-04


-----

### Training/Testing

In the previous examples, we used *all* of the data to fit the regressor
and compute an accuracy. In practice that is not a good idea, since we
will have little idea how well our regressor will perform on new, unseen
data. A better idea is to train on a subset of the data and test this
new regressor on unseen _test_ data. We do this below, by splitting our
data into a _training_ sample, and a testing sample by using the
`train_test_split` method in scikit learn. Specifically in this example,
we use 75% of the data for training and 25% of the data for testing (you
should change this value and see how the scores change). In this case,
our regressor achieves nearly the same score as the full data regressor
achieved, indicating that we have a fairly robust predictor.

-----

In [10]:
from sklearn.cross_validation import train_test_split
from sklearn import metrics

# Evaluate the model by splitting into train and test sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

# Fit a new model
model2 = LogisticRegression()
model2.fit(x_train, y_train)

predicted = model2.predict(x_test)
probs = model2.predict_proba(x_test)

# Generate and display different evaluation metrics
print(metrics.accuracy_score(y_test, predicted))
print( metrics.roc_auc_score(y_test, probs[:, 1]))

print(metrics.confusion_matrix(y_test, predicted))
print(metrics.classification_report(y_test, predicted))

0.833041817077
0.854490188138
[[49446  2488]
 [10971 17708]]
             precision    recall  f1-score   support

        0.0       0.82      0.95      0.88     51934
        1.0       0.88      0.62      0.72     28679

avg / total       0.84      0.83      0.82     80613



-----

We can also display more traditional measures of the regressor's
accuracy. In this case, we can compute and display the mean absolute
error, the mean squared error, and the root mean square error.

-----

In [11]:
print("MAE = {:5.4f}".format(metrics.mean_absolute_error(y_test, predicted)))
print("MSE = {:5.4f}".format(metrics.mean_squared_error(y_test, predicted)))
print("RMSE = {:5.4f}".format(np.sqrt(metrics.mean_squared_error(y_test, predicted))))

MAE = 0.1670
MSE = 0.1670
RMSE = 0.4086


----

One last point is that we can obtain an even better understanding of the
accuracy of our regression by repeatedly testing how well this approach
works with random subsets of the original data. We can do this by using
the `cross_validation` functionality in the scikit learn library, as
shown below. In this case, we compute the regressor for ten different
subsets, and compute the mean score, which demonstrates the power of
this approach.

-----

In [12]:
from sklearn.cross_validation import cross_val_score

# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), x, y, scoring='accuracy', cv=10)
print(scores)
print(scores.mean())


[ 0.8331576   0.84329839  0.84137567  0.84283322  0.83973206  0.84512327
  0.84413087  0.83240293  0.81686515  0.80864657]
0.834756574718
